In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022-2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import argparse
import sys
import json

import multiprocessing as mp



parser = argparse.ArgumentParser()
parser.add_argument("-c", "--config", type=str, default="Example_highRes", help="Name of radar config.")
args, _ = parser.parse_known_args()
from isaacsim import SimulationApp

# Example for creating a RTX lidar sensor and publishing PCL data
simulation_app = SimulationApp({"headless": True})


Starting kit application with the following args: [Warning] [simulation_app.simulation_app] fast shutdown not supported with jupyter notebooks
 ['/home/ronim/isaacsim/_build/linux-x86_64/release/exts/isaacsim.simulation_app/isaacsim/simulation_app/simulation_app.py', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps/isaacsim.exp.base.python.kit', '--/app/tokens/exe-path=/home/ronim/isaacsim/_build/linux-x86_64/release/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--/app/installSignalHandlers=0', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/exts', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps', '--/physics/cudaDe

[9.442s] Simulation App Startup Complete
2025-07-21T12:30:37Z [18,607ms] [Warning] [omni.fabric.plugin] USD->Fabric: Unhandled array type string[]
2025-07-21T12:30:37Z [18,611ms] [Warning] [usdrt.population.plugin] [UsdNoticeHandler] Unhandled attribute type VtArray<std::string> (prim attribute: omni:rtx:material:db:flattener:transmittance_color)
2025-07-21T12:30:37Z [18,611ms] [Warning] [usdrt.population.plugin] [UsdNoticeHandler] Unhandled attribute type VtArray<std::string> (prim attribute: omni:rtx:material:db:flattener:reflection_roughness_constant)
2025-07-21T12:30:37Z [18,611ms] [Warning] [usdrt.population.plugin] [UsdNoticeHandler] Unhandled attribute type VtArray<std::string> (prim attribute: omni:rtx:material:db:flattener:ior_constant)
2025-07-21T12:30:37Z [18,662ms] [Warning] [omni.sensors.nv.radar.wpm_dmatapprox.plugin] MotionBVH activation state 0 doesn't match requested state 1


2025-07-21T12:30:37Z [18,662ms] [Fatal] [omni.sensors.nv.radar.wpm_dmatapprox.plugin] Running radar without MotionBVH is disallowed, to force it use --/app/sensors/nv/radar/runWithoutMBVH=true
If you are running on Windows and have motionBVH enabled, be sure to enable Vulkan as well by passing --vulkan
terminate called without an active exception
2025-07-21T12:30:37Z s] [Warning] [carb.crashreporter-breakpad.plugin] [crash] A crash has occurred.  If a debugger should be attached, please set the '/crashreporter/debuggerAttachTimeoutMs' setting to a timeout in milliseconds.  This can be used to allow the crash reporter to wait for up to that long for a debugger to attach before processing or sending the crash report.
2025-07-21T12:30:37Z s] [Error] [carb.crashreporter-breakpad.plugin] [crash] Wrote dump file '/home/ronim/.cache/packman/chk/kit-kernel/107.3.0+feature.199947.b0a86421.gl.manylinux_2_35_x86_64.release/data/Kit/Isaac-Sim Python/5.0/21b26ebc-a9b7-41f8-5c35629b-2d081230.dmp'
20

In [2]:



import carb
import omni
import omni.kit.viewport.utility
import omni.replicator.core as rep
from isaacsim.core.api import SimulationContext
from isaacsim.core.utils import stage
from isaacsim.core.utils.extensions import enable_extension
from isaacsim.storage.native import get_assets_root_path
from pxr import Gf
from isaacsim.core.api.objects import DynamicCuboid
from isaacsim.core.api import World
from isaacsim.core.prims import RigidPrim
import matplotlib.pyplot as plt
# enable ROS bridge extension
enable_extension("isaacsim.util.debug_draw")

simulation_app.update()

# Locate Isaac Sim assets folder to load environment and robot stages
assets_root_path = get_assets_root_path() 
if assets_root_path is None:
    carb.log_error("Could not find Isaac Sim assets folder")
    simulation_app.close()
    sys.exit()

simulation_app.update()
world = World()
dynamic_cube = DynamicCuboid(prim_path="/World/cube", translation=(0, 5, 1), scale=(10, 10, 10))
rigid_cube = RigidPrim("/World/cube")
world.scene.add_default_ground_plane()


simulation_app.update()
sensor_attributes = {'omni:sensor:WpmDmat:scan:s001:maxRangeM': 50}

# Create the radar sensor that generates data into "RtxSensorCpu"
# Sensor needs to be rotated 90 degrees about +Z so it faces warehouse shelves.
# Possible config options are Example.
_, sensor = omni.kit.commands.execute(
    "IsaacSensorCreateRtxRadar",
    path="/sensor",
    parent=None,
    translation=(0, 0, 1.0),
    orientation=Gf.Quatd(0.70711, 0.0, 0.0, 0.70711),
    **sensor_attributes
)
# hydra_texture = rep.create.render_product(sensor.GetPath(), [1, 1], name="Isaac")

render_product = rep.create.render_product(sensor.GetPath(), [1, 1], name="RtxRadarOutput")

annotator = rep.AnnotatorRegistry.get_annotator("IsaacExtractRTXSensorPointCloudNoAccumulator")
annotator.initialize()

# render_product_path = '/sensor'
simulation_context = SimulationContext(physics_dt=1.0 / 60.0, rendering_dt=1.0 / 60.0, stage_units_in_meters=1.0)
simulation_app.update()
annotator.attach([render_product])

# Create the debug draw pipeline in the post process graph
# writer = rep.writers.get("RtxRadar" + "DebugDrawPointCloud")
# writer.attach([hydra_texture])

simulation_app.update()

simulation_context.play()
world.reset()

# def plot_scatter(queue):
#     import matplotlib
#     matplotlib.use("Agg")
#     import matplotlib.pyplot as plt


    

#     while True:
#         data = queue.get(timeout=0.1)
#         if data is None:
#                 break
#         plot.set_xdata(data[:, 0])
#         plot.set_ydata(data[:, 1])
#         plt.draw()
#         plt.pause(0.001)
# queue = mp.Queue()

# # start encoding thread - this will encode frames from the queue and send them to the RTSP stream
# plot_thread = mp.Process(target=plot_scatter, args = (queue,))
# plot_thread.start()

# while simulation_app.is_running():
#     simulation_app.update()
#     radar_data = annotator.get_data()
#     queue.put(radar_data['data'])
#     print(radar_data['data'])


# # cleanup and shutdown
# simulation_context.stop()
# simulation_app.close()


: 

In [ ]:
plt.ion()
fig, ax = plt.subplots()
plot, = ax.plot([], [],'*')
while simulation_app.is_running():
    simulation_app.update()
    radar_data = annotator.get_data()
    plot.set_xdata(data[:, 0])
    plot.set_ydata(data[:, 1])
    plt.draw()
    plt.pause(0.001)
    print(radar_data['data'])


# cleanup and shutdown
simulation_context.stop()
simulation_app.close()
